In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os
import pandas as pd

Using TensorFlow backend.


In [2]:
model = load_model('HandEmo.h5')

In [3]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis



In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))

    return max(pred_probab), pred_class

def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [ ]:
emojis = get_emojis()
cap = cv2.VideoCapture(0)
x,y,w,h = 300, 50, 350, 350
while(cap.isOpened()):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
    res = cv2.bitwise_and(img, img , mask=mask2)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    median = cv2.GaussianBlur(gray, (5,5), 0)

    kernel_square = np.ones((5,5), np.uint8)
    dilation = cv2.dilate(median, kernel_square, iterations=2)
    opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
    ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

    thresh = thresh[y:y+h , x:x+w]
    contours = cv2.findContours(thresh.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour)>2500:
            x, y, w1, h1= cv2.boundingRect(contour)
            newImage = thresh[y:y+h1 , x:x+w1]
            newImage = cv2.resize(newImage, (50,50))
            pred_probab, pred_class = keras_predict(model, newImage)
            print(pred_class, pred_probab)
            img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

    x,y,w,h = 300, 50, 350,350
    cv2.imshow('Frame',img)
    cv2.imshow('Contours',thresh)
    k = cv2.waitKey(10)
    if k==27:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
1 0.97430134
11 0.49052113
11 0.49052113
3 0.6662943
3 0.6662943
11 0.2953927
11 0.2953927
11 0.6884599
11 0.6884599
2 0.9835554
2 0.9835554
5 0.75804317
4 0.6661737
4 0.83536613
4 0.83536613
4 0.73235935
4 0.73235935
4 0.9652829
4 0.9028745
4 0.9028745
4 0.9409913
4 0.98738354
4 0.9503571
4 0.9107754
4 0.9107754
4 0.8241489
4 0.9438582
4 0.9438582
7 0.7588506
7 0.99987924
7 0.99987924
7 0.8621886
7 0.9847439
7 0.9847439
7 0.7676345
7 0.6309777
7 0.99763966
7 0.85961545
7 0.95903736
7 0.95903736
7 0.99978346
7 0.99978346
7 0.9983456
7 0.9992823
7 0.9974432
7 0.9974432
2 0.28514367
2 0.28514367
7 0.93677276
7 0.93677276
7 0.99301964
7 0.99301964
7 0.42652872
7 0.9986726
7 0.9986726
7 0.99427235
7 0.99427235
7 0.9999423
7 0.9999423
7 0.99979895
7 0.99979895
7 0.9998093
7 0.9998093
7 0.9983816
7 0.9209715
7 0.9209715
7 0.33972448
7 0.99826956
7 0.99826956
7 0.9996131
7 0.9996131
7 0.63238114
7 0.63238114
7 0.9998386
5 0.7517012
5 0.7517012
7 0.9985312
7 0.9985

5 0.8235302
5 0.8697002
5 0.76915014
5 0.7610818
4 0.42477286
4 0.42477286
2 0.256882
2 0.256882
5 0.8318241
5 0.8933769
5 0.96361536
5 0.96361536
5 0.9805772
5 0.9883237
5 0.9922805
5 0.9922805
5 0.9961797
5 0.9961797
5 0.99890697
5 0.99940896
5 0.99940896
5 0.9993493
5 0.99932075
5 0.99932075
5 0.9991035
5 0.9991035
5 0.9992773
5 0.9992773
5 0.9994392
5 0.9993845
5 0.9993845
5 0.9994837
5 0.9996706
5 0.9996706
5 0.99976
5 0.99877244
5 0.9996239
5 0.9996239
5 0.9997521
5 0.99970645
5 0.99970645
5 0.99971706
5 0.9997086
5 0.9997166
5 0.9997166
5 0.99969494
5 0.99901676
5 0.99901676
1 0.92607075
1 0.9180654
1 0.85801655
1 0.85801655
1 0.54429924
1 0.88090324
1 0.8514489
1 0.8514489
1 0.9513207
1 0.84067017
1 0.84764385
1 0.84764385
1 0.8432271
1 0.939766
1 0.939766
1 0.8768201
1 0.94556946
1 0.94157463
1 0.9201539
1 0.9201539
1 0.91646945
1 0.948085
1 0.9604928
1 0.91044074
1 0.91044074
1 0.71290165
11 0.8700558
11 0.8700558
11 0.6949936
11 0.61775184
11 0.8809296
11 0.93745774
11 0.937

9 0.84525585
9 0.84525585
9 0.7956847
11 0.7107651
11 0.7107651
1 0.644518
1 0.644518
9 0.5502324
11 0.80082345
11 0.80082345
9 0.44656327
9 0.44656327
9 0.59243035
9 0.8156179
9 0.8156179
11 0.7740388
11 0.7740388
1 0.5084645
11 0.7326936
1 0.7092493
9 0.5947479
9 0.5947479
11 0.8333465
9 0.59257543
9 0.5696174
11 0.58803695
11 0.8555689
11 0.53623855
11 0.53623855
9 0.8795642
9 0.8795642
11 0.8290535
11 0.47568005
11 0.7050396
11 0.7050396
11 0.45644078
3 0.41170573
11 0.49300244
11 0.49300244
9 0.80089587
11 0.45416674
11 0.6509298
9 0.50309795
11 0.7825332
11 0.7825332
1 0.4523915
1 0.4523915
11 0.8220812
9 0.4905801
9 0.4905801
11 0.84952205
11 0.90172017
1 0.6529669
11 0.55564415
11 0.69596046
11 0.48606393
11 0.5654428
11 0.5654428
4 0.28236905
4 0.28236905
11 0.6790529
11 0.6790529
11 0.84445703
3 0.64246905
11 0.5915595
11 0.5915595
3 0.54084706
3 0.54084706
3 0.62515354
3 0.62515354
11 0.59271103
11 0.59271103
3 0.49038005
3 0.49038005
11 0.52126706
11 0.52126706
3 0.46876198

11 0.57661456
9 0.23831135
9 0.23831135
3 0.39818266
9 0.26434907
9 0.26434907
4 0.29260853
4 0.29260853
9 0.24357483
9 0.24357483
11 0.25103173
3 0.30371523
3 0.30371523
3 0.40762916
4 0.39150193
4 0.39150193
10 0.23321365
10 0.23321365
4 0.4174443
4 0.4174443
10 0.2419181
10 0.38299724
10 0.38299724
4 0.3492116
3 0.27351332
3 0.27351332
9 0.25710002
3 0.39605233
3 0.39605233
7 0.2871129
3 0.30057675
3 0.30057675
4 0.29406843
9 0.2292237
9 0.2292237
7 0.32868597
7 0.32868597
4 0.21303323
4 0.21303323
11 0.4349495
10 0.8452635
10 0.8452635
7 0.514709
10 0.27338144
4 0.31891835
4 0.31891835
7 0.33341932
7 0.33341932
3 0.5019399
11 0.48344716
10 0.6424428
6 0.5554991
6 0.5554991
4 0.42338097
10 0.6792164
10 0.6792164
7 0.4332635
7 0.27364945
7 0.27364945
11 0.32951158
11 0.32951158
7 0.33946708
7 0.33946708
3 0.25255242
3 0.25255242
3 0.31355518
3 0.31355518
3 0.4460997
10 0.34233096
10 0.34233096
10 0.29250708
10 0.29250708
11 0.38457137
11 0.38457137
11 0.3742852
11 0.3742852
4 0.43037

1 0.6000221
10 0.48875606
10 0.48875606
11 0.2904358
11 0.2904358
9 0.46859035
9 0.46859035
10 0.5500464
10 0.5500464
10 0.45029768
10 0.6179889
10 0.6179889
10 0.97338337
10 0.97338337
10 0.9948338
10 0.9992607
10 0.9992607
10 0.96125907
10 0.9439895
10 0.9439895
10 0.9935121
10 0.9935121
10 0.97188133
10 0.97188133
10 0.9945966
10 0.9979463
10 0.9979463
10 0.99724543
10 0.99724543
10 0.9997074
10 0.9997074
10 0.8155929
10 0.8155929
10 0.95973206
10 0.95973206
10 0.9978523
10 0.99732375
10 0.99732375
10 0.9980521
10 0.9980521
10 0.99486613
10 0.99486613
10 0.9962625
10 0.9962625
10 0.99861753
10 0.9950518
10 0.9950518
10 0.9686072
10 0.9686072
7 0.46124575
1 0.99865
1 0.99865
10 0.7698372
10 0.7698372
10 0.98598236
10 0.99957055
10 0.99839056
10 0.99712664
10 0.9883696
10 0.9883696
10 0.96202666
10 0.94841766
10 0.94841766
10 0.9836447
10 0.9836447
10 0.9915258
10 0.9915258
10 0.9975038
2 0.7887805
10 0.994599
10 0.994599
10 0.84301966
10 0.84301966
10 0.7676571
10 0.7676571
7 0.97552

4 0.99769944
4 0.99769944
4 0.9937523
4 0.9987772
4 0.9987772
4 0.9957034
4 0.9955058
4 0.9989435
4 0.9989435
4 0.999338
4 0.99729013
4 0.99729013
4 0.9975199
4 0.9983298
4 0.9983298
4 0.8371705
9 0.61962277
9 0.61962277
9 0.6600385
9 0.7825104
9 0.7825104
9 0.85818726
9 0.85818726
11 0.93264955
11 0.93264955
11 0.95238197
11 0.95238197
11 0.9162485
11 0.9162485
11 0.9861398
11 0.9861398
11 0.9448681
11 0.9448681
11 0.9351506
11 0.9351506
8 0.5036487
8 0.5036487
8 0.60934913
8 0.60934913
8 0.60424703
8 0.60424703
1 0.763439
1 0.763439
1 0.6680868
1 0.36965752
1 0.36965752
1 0.7999281
8 0.48776728
8 0.48776728
8 0.39902174
1 0.36045316
1 0.36045316
1 0.41197658
8 0.42347834
8 0.42347834
8 0.5166735
1 0.820971
1 0.90833163
1 0.90833163
1 0.6322862
1 0.99619067
1 0.99619067
1 0.9878015
1 0.9878015
1 0.9981128
1 0.9978319
1 0.9923504
1 0.9923504
1 0.99257636
1 0.9928651
11 0.9900731
11 0.9900731
11 0.97918624
11 0.97918624
11 0.94088036
11 0.94088036
11 0.9323545
11 0.9323545
11 0.92807806

4 0.7947839
4 0.7947839
4 0.86770344
4 0.86770344
7 0.50527793
7 0.50527793
4 0.80366564
4 0.80366564
4 0.8114859
4 0.8114859
4 0.7986745
4 0.7986745
4 0.7947524
4 0.7947524
4 0.68966645
4 0.68966645
4 0.532568
4 0.532568
7 0.7862911
7 0.7862911
7 0.8609743
7 0.8609743
7 0.47586283
7 0.47586283
7 0.7136384
7 0.7136384
7 0.99612135
7 0.98201203
7 0.98201203
4 0.35963526
4 0.35963526
9 0.3158699
9 0.3158699
7 0.54946893
7 0.54946893
7 0.94592065
7 0.94592065
5 0.99997807
5 0.99997807
5 0.9140309
5 0.9140309
5 0.9882386
5 0.9882386
5 0.8988107
5 0.8988107
5 0.49450698
4 0.76017517
4 0.76017517
4 0.91185033
4 0.91185033
4 0.864431
4 0.8148448
4 0.8148448
4 0.80641484
4 0.80641484
4 0.66510785
4 0.56994367
4 0.56994367
3 0.58998966
3 0.77454096
3 0.77454096
3 0.75995964
3 0.6361742
3 0.6361742
3 0.5103281
4 0.63995993
4 0.63995993
4 0.6216676
4 0.6216676
4 0.4008989
4 0.29212824
4 0.29212824
7 0.3142591
7 0.3142591
7 0.33602557
7 0.33602557
7 0.38003755
7 0.38003755
4 0.36144072
4 0.3614407